# <span style="color: blue;">Система импорта</span>

### И снова оператор import

Что происходит в момент исполнения оператора `import`?

In [1]:
import dis

dis.dis("import useful")

  1           0 LOAD_CONST               0 (0)
              2 LOAD_CONST               1 (None)
              4 IMPORT_NAME              0 (useful)
              6 STORE_NAME               0 (useful)
              8 LOAD_CONST               1 (None)
             10 RETURN_VALUE


Инструкция `IMPORT_NAME` вызывает встроенную функцию `__import__`:

In [ ]:
useful = __import__("useful", globals(), None, None, 0)

Поиск функции "`__import__`" в "`builtins`" происходит динамически, а значит можно применить метод **monkey patching**

### Трассировка импортов

In [2]:
### НЕ ЗАПУСКАТЬ ЭТОТ ПРИМЕР :-)

def import_wrapper(name, *args, std_import=__import__):
    print("importing ", name)
    return std_import(name, *args)

import builtins
builtins.__import__ = import_wrapper

import collections

importing  collections


Будет выведено:

Параметр `std_import` хитро используется для сохранения изначального значения `__import__`

На практике для импорта модуля по имени следует использовать функцию `import_module` из пакета `importlib`:

In [1]:
import importlib

importlib.import_module("useful")

<module 'useful' from '/home/ubuntu/projects/labs.in.ua/anaconda.dev/useful.py'>

### Внутри функции `__import__`

Что же всё-таки делает функция `__import__`?

Сначала для модуля создаётся (пустой) объект.

In [4]:
import types

mod = types.ModuleType("useful")

Затем байт код модуля вычисляется в пространстве имён созданного объекта:

In [8]:
# загружаем исходный код модуля
with open("./useful.py") as handle:
    source = handle.read()

# компилируем исходник и получаем байт-код:
code = compile(source, "useful.py", mode="exec")

# выполнение байт-кода (передавая туда globals)
exec(code, mod.__dict__)

dir(mod)

['__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'boo',
 'some_variable',
 'test']

В завершение объект присваивается переменной с соответсвующим именем:

In [6]:
useful = mod
useful  # ≈ import useful

<module 'useful'>

### Компиляция модулей _(первый этап кеширования)_

При первом импорте исходный код модуля компилируется в байткод, который кешируется в файле с расширением **`.pyc`**

In [10]:
def read_int(handle):
    return int.from_bytes(handle.read(4), "little")

import useful
useful.__cached__

'/home/ubuntu/projects/labs.in.ua/anaconda.dev/__pycache__/useful.cpython-36.pyc'

In [14]:
handle = open(useful.__cached__, "rb")
magic = read_int(handle)  # "3310\r\n" для 3.4
mtime = read_int(handle)  # дата последнего изменения

In [15]:
import time
time.asctime(time.localtime(mtime))

'Thu Oct 19 02:30:58 2017'

In [16]:
read_int(handle)  # размер файла

203

In [17]:
import marshal

marshal.loads(handle.read())

<code object <module> at 0xb294b020, file "/home/ubuntu/projects/labs.in.ua/anaconda.dev/useful.py", line 1>

### sys.modules _(второй этап кеширования)_

Полученный в результате импорта модуль попадает в специальный словарь **`sys.modules`**.

Ключом в словаре является имя модуля, то есть значение атрибута **`__name__`**:

In [18]:
import useful
import sys

"useful" in sys.modules

True

In [ ]:
list(sys.modules)

Повторный импорт уже загруженного модуля не приводит к его перезагрузке:

In [19]:
id(sys.modules["useful"])

2996006100

In [20]:
import useful

id(sys.modules["useful"])

2996006100

**Важно:** Учитывайте это при работе в интерактивном режиме!<br/>
_(изменённые версии модулей переподгружаться не будут)_

### Циклические импорты

Мотивирующий пример:

In [ ]:
# useful/__init__.py:

from .foo import *

some_variable = 42

In [ ]:
# useful/foo.py:

from . import some_variable

def foo():
    print(some_variable)

Попробуем импортировать пакет `useful`:

In [ ]:
import useful

Получим ошибку:

### Циклические импорты: как решить?

Бороться с циклическими импортами можно, как минимум, тремя способами.

**1).** Вынести общую функциональность в отдельный модуль.

In [25]:
# useful/_common.py

some_variable = 42

**2).** Сделать импорт локальным для использующих его функций/методов:

In [26]:
# useful/foo.py:

def foo():
    from . import some_variable
    
    print(some_variable)

**3).** Пойти наперекор `PEP-8` и изменить модуль `__init__.py` так, чтобы импорт происходил в конце модуля:

In [ ]:
# useful/__init__.py:

some_variable = 42

from .foo import *

Но так лучше никогда не делайте!!

### Подробнее о sys.path

Напоминание:
* `sys.path` — список “путей”, в которых Python ищет модули и пакеты при импорте
* “путь” — произвольная строка, например, директория или zip-архив
* импортировать можно только то, что доступно через `sys.path`

При импорте модуля обход `sys.path` происходит слева направо до тех пор, пока модуль не будет найден:

In [28]:
import sys

sys.path

['',
 '/home/ubuntu/anaconda3/lib/python36.zip',
 '/home/ubuntu/anaconda3/lib/python3.6',
 '/home/ubuntu/anaconda3/lib/python3.6/lib-dynload',
 '/home/ubuntu/anaconda3/lib/python3.6/site-packages',
 '/home/ubuntu/anaconda3/lib/python3.6/site-packages/Sphinx-1.5.6-py3.6.egg',
 '/home/ubuntu/anaconda3/lib/python3.6/site-packages/setuptools-27.2.0-py3.6.egg',
 '/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/home/ubuntu/.ipython']

Первый параметр -- это текущая директория

In [29]:
open("collections.py", "w").close()

import collections
collections  # теперь это локальный модуль! :(

<module 'collections' from '/home/ubuntu/anaconda3/lib/python3.6/collections/__init__.py'>

Мораль: **никогда** не называйте свои модули как модули стандартной библиотеки.

### Построение sys.path

При старте интерпретатора в `sys.path` находятся текущая директория и директории стандартной библиотеки:

In [ ]:
$ python3 -S -c 'import sys; print(sys.path)'
['', '/usr/lib/python3.6/', ...]

Затем к ним добавляются директории с пакетами, установленными пользователем:

In [ ]:
$ python3 -c 'import sys; print(sys.path)'
['', ..., '/usr/lib/python3.6/site-packages', ...]

Директории, перечисленные в переменной окружения `PYTHONPATH`, попадают в начало `sys.path`:

In [ ]:
$ PYTHONPATH=foo:bar python3 -c 'import sys; print(sys.path)'
['', './foo', './bar', ...]

Кроме того, `sys.path` можно изменять программно:

In [ ]:
import sys

sys.path.extend(["foo", "bar"])

### sys.path и sys.meta_path

Может показаться, что `sys.path` — властелин и повелитель импорта, но это не так.

Управляет импортом **`sys.meta_path`**:

In [31]:
import sys

sys.meta_path

[_frozen_importlib.BuiltinImporter,
 _frozen_importlib.FrozenImporter,
 _frozen_importlib_external.PathFinder,

**Импортер** — экземпляр класса, реализующего протоколы **искателя** (aka `Finder`) и **загрузчика** (aka `Loader`):
* **`Finder`** любым известным ему способом ищет модуль
* **`Loader`** загружает то, что **`Finder`** нашёл

Если класс реализует оба протокола, он называется **импортером**.

**`BuiltinImporter`** загружает встроенные модули, например `sys`:

In [2]:
import sys
sys

<module 'sys' (built-in)>

**`FrozenImporter`** загружает модули, уже скомпилированные "нативно".

**`PathFinder`** — стандартный поиск модулей по `sys.path`

### Протокол искателя: Finder

**`Finder`** должен реализовывать метод **`find_spec`**:
* принимающий имя модуля
* возвращающий `ModuleSpec` (или `None`, если модуль не был найден)

In [3]:
builtin_finder = sys.meta_path[0]
path_finder = sys.meta_path[2]

builtin_finder.find_spec("itertools")

ModuleSpec(name='itertools', loader=<class '_frozen_importlib.BuiltinImporter'>, origin='built-in')

In [37]:
builtin_finder.find_spec("enum")

In [38]:
path_finder.find_spec("enum")

ModuleSpec(name='enum', loader=<_frozen_importlib_external.SourceFileLoader object at 0xb3b40bec>, origin='/home/ubuntu/anaconda3/lib/python3.6/enum.py')

In [39]:
path_finder.find_spec("math")

ModuleSpec(name='math', loader=<_frozen_importlib_external.ExtensionFileLoader object at 0xb15c74cc>, origin='/home/ubuntu/anaconda3/lib/python3.6/lib-dynload/math.cpython-36m-i386-linux-gnu.so')

### Протокол искателя: ModuleSpec

`ModuleSpec` содержит всю необходимую для загрузки информацию о модуле:

In [4]:
spec = path_finder.find_spec("collections")
spec.name

'collections'

In [5]:
spec.origin

'/home/ubuntu/anaconda3/lib/python3.6/collections/__init__.py'

In [6]:
spec.cached

'/home/ubuntu/anaconda3/lib/python3.6/collections/__pycache__/__init__.cpython-36.pyc'

In [7]:
spec.parent

'collections'

In [8]:
spec.submodule_search_locations

['/home/ubuntu/anaconda3/lib/python3.6/collections']

In [9]:
spec.loader

### Модернизация модулей

**Мотивация:** у нескольких модулей одинаковый интерфейс и отличаются они, например, скоростью работы.

Хочется попробовать импортировать более быстрый, а в случае ошибки использовать медленный.

In [10]:
try:
    import _useful_speedups as useful
except ImportError:
    import useful

Более надёжный вариант использует функцию `find_spec` из модуля `importlib.util`:

In [12]:
from importlib.util import find_spec

if find_spec("_useful_speedups"):
    import _useful_speedups as useful
else:
    import useful

Функция `find_spec` обходит `sys.meta_path` и последовательно вызывает одноимённый метод у каждого из импортеров, пока не найдёт модуль.

### Протокол загрузчика: Loader

`Loader` должен реализовывать два метода `create_module` для создания пустого модуля и `exec_module` для его заполнения.

In [13]:
spec = find_spec("enum")

mod = spec.loader.create_module(spec)
mod  # None -- используем стандартный загрузчик.

In [14]:
from importlib.util import module_from_spec

mod = module_from_spec(spec)
mod

<module 'enum' from '/home/ubuntu/anaconda3/lib/python3.6/enum.py'>

In [15]:
dir(mod)

['__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__']

In [16]:
spec.loader.exec_module(mod)

dir(mod)

['DynamicClassAttribute',
 'Enum',
 'EnumMeta',
 'Flag',
 'IntEnum',
 'IntFlag',
 'MappingProxyType',
 'OrderedDict',
 '_EnumDict',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_auto_null',
 '_decompose',
 '_high_bit',
 '_is_descriptor',
 '_is_dunder',
 '_is_sunder',
 '_make_class_unpicklable',
 '_or_',
 '_power_of_two',
 '_reduce_ex_by_name',
 'auto',
 'reduce',
 'sys',
 'unique']

### Пример: автоматическая установка

In [2]:
import sys
import subprocess
from importlib.util import find_spec
from importlib.abc import MetaPathFinder

class AutoInstall(MetaPathFinder):
    _loaded = set()
    
    @classmethod
    def find_spec(cls, name, path=None, target=None):
        if path is not None or name in cls._loaded:
            return None
        print("Installing", name)
        cls._loaded.add(name)
        try:
            subprocess.check_output(
                [sys.executable, "-m", 
                 "pip", "install", name]
            )
            return find_spec(name)
        except Exception:
            print("Failed")

sys.meta_path.append(AutoInstall)

**Вопрос:** Для чего атрибут `_loaded`?

### Знакомьтесь, sys.path_hooks!

К сожалению, на `sys.meta_path`, искателях и загрузчиках история не заканчивается.

В игру вступает **`sys.path_hooks`** — ещё один список, используемый искателем `PathFinder`.

В `sys.path_hooks` находятся функции, задача которых: подобрать каждому элементу `sys.path` — искателя.

In [17]:
sys.path_hooks

[zipimport.zipimporter,
 <function _frozen_importlib_external.FileFinder.path_hook.<locals>.path_hook_for_FileFinder>]

In [22]:
sys.path_hooks[0]('/usr/lib/python3')

ZipImportError: not a Zip file

In [21]:
sys.path_hooks[1]('/usr/lib/python3')

FileFinder('/usr/lib/python3')

### Пример: удалённый импорт

In [ ]:
import re
import sys
from urllib.request import urlopen

def url_hook(url):
    if not url.startswith(("http://", "https://")):
        raise ImportError
    with urlopen(url) as page:
        data = page.read().decode("utf-8")
    filenames = re.findall("[a-zA-Z_][a-zA-Z0-9_]*.py", data)
    module_names = {name[:-3] for name in filenames}  # отрезаем ".py"
    return URLFinder(url, module_names)

sys.path_hooks.append(url_hook)

### Удаленный импорт: URLFinder

In [ ]:
from importlib.abc import PathEntryFinder
from importlib.utils import spec_from_loader

class URLFinder(PathEntryFinder):
    def __init__(self, url, available):
        self.url = url
        self.available = available
        
    def find_spec(self, name, target=None):
        if name not in self.available:
            return None
        loader = URLLoader()
        origin = "{}/{}.py".format(self.url, name)
        return spec_from_loader(name, loader, origin=origin)           

### Удаленный импорт: URLLoader

In [ ]:
from urllib.request import urlopen

class URLLoader:
    def create_module(self, target):
        return None
    
    def exec_module(self, module):
        with urlopen(module.__spec__.origin) as page:
            source = page.read()
        code = compile(source, 
                       module.__spec__.origin,
                       mode="exec")
        exec(code, module.__dict__)

### Удаленный импорт в действии

Модуль, который будем удалённо загружать:

In [ ]:
# remote.py:

print("It works!")

Запускаем простой веб-сервер из папки с `remote.py`

In [ ]:
$ python -m http.server
Serving HTTP on 0.0.0.0 port 8000 ...

То, что возвращает веб-сервер:

In [ ]:
$ curl http://127.0.0.1:8000/
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" 
    "http://www.w3.org/TR/html4/strict.dtd">
<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=ascii">
<title>Directory listing for /</title>
</head>
<body>
<h1>Directory listing for /</h1>
<hr>
<ul>
<li><a href="remote.py">remote.py</a></li>
</ul>
<hr>
</body>
</html>

Пусть весь пример выше расположен в `activate.py`. Запустим интерактивный `python`:

In [ ]:
import remote

Будет выведено:

Добавим наш URL в `sys.path` и попробуем импортировать снова:

In [ ]:
import sys
sys.path.append("http://localhost:8000")

import remote

Получим:

### Система импорта: резюме

Система импорта нетривиальна.

Импорт контролируется импортерами, задача которых — найти модуль по имени и загрузить его.

После загрузки интерпретатора в `sys.meta_path` появляются импортеры для работы со встроенными модулями, а также модулями в zip-архивах и “путях”.

“Путевой” искатель aka `PathFinder` можно расширять, добавляя новые “пути” к `sys.path` и функции к `sys.path_hooks`.